<a href="https://colab.research.google.com/github/ccitChun/1081ML/blob/main/one_cnn_ks3_pool4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install thop

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from torchvision import datasets
import numpy as np
from tqdm import tqdm
from google.colab import drive
from sklearn.metrics import precision_score, recall_score, f1_score
from thop import profile  # for calculating FLOPS and parameters

In [3]:
drive.mount('/content/drive')
os.getcwd()
os.chdir("/content/drive/MyDrive") #更改路徑
os.listdir()
folders = os.listdir('./data2')
folders.sort()
data_path = '/content/drive/MyDrive/data2'
file_paths = []
labels = []

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Dataset class
class MyDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths# 存儲圖像文件路徑的列表
        self.labels = labels # 與圖像對應的標籤列表
        self.transform = transform # 可選的圖像變換（例如數據增強）

    def __len__(self):
        return len(self.file_paths) # 返回數據集中的圖像總數

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]  # 獲取索引為idx的圖像路徑
        image = Image.open(img_path).convert('RGB') # 打開圖像並確保它是RGB格式
        label = self.labels[idx] # 獲取相應的標籤

        if self.transform:
            image = self.transform(image) # 如果指定了轉換，則應用它

        return image, label # 返回圖像和標籤的元組

# Setup transformations
train_transform = transforms.Compose([
    transforms.Resize((128, 128),interpolation=Image.BILINEAR), # 將圖像大小調整為224x224像素
   # transforms.RandomHorizontalFlip(), # 隨機水平翻轉圖像（數據增強）
    transforms.ToTensor(), # 將圖像轉換為PyTorch張量
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # 歸一化圖像
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128), interpolation=Image.BILINEAR),  # 同樣將圖像大小調整為224x224像素
    transforms.ToTensor(), # 將圖像轉換為PyTorch張量
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # 同樣歸一化圖像
])

In [5]:
for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_paths.append(os.path.join(folder_path, file_name))
            labels.append(folder_name)

In [6]:
# Define the lightweight CNN
class LightweightCNN(nn.Module):
    def __init__(self, num_classes):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, padding='same')
        self.pool1 = nn.MaxPool2d(4)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(10 * 32 * 32, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [7]:
# Convert labels to numerical values
label_to_index = {label: idx for idx, label in enumerate(sorted(set(labels)))}
labels = [label_to_index[label] for label in labels]

In [8]:
# Split data
train_files, test_files, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

train_dataset = MyDataset(train_files, train_labels, transform=train_transform)
test_dataset = MyDataset(test_files, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [9]:
# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LightweightCNN(num_classes=len(label_to_index)).to(device)

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1

In [12]:
# Training and evaluation function
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.to(device)

    # Calculate FLOPS and parameters
    input_tensor = torch.randn(1, 3, 128, 128).to(device)
    flops, params = profile(model, inputs=(input_tensor, ))
    flops_m = flops / 10**6
    params_m = params / 10**6
    print(f'FLOPS: {flops_m:.6f} MFLOPs, Parameters: {params_m:.6f} M')

    for epoch in range(num_epochs):
        model.train()
        total_loss, total_correct = 0, 0
        y_true, y_pred = [], []

        # Iterate over the training data
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.argmax(1).cpu().numpy())

        train_loss = total_loss / len(train_loader.dataset)
        train_accuracy = total_correct / len(train_loader.dataset)
        train_precision, train_recall, train_f1 = calculate_metrics(y_true, y_pred)

        model.eval()
        total_loss, total_correct = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                total_correct += (outputs.argmax(1) == labels).sum().item()

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(outputs.argmax(1).cpu().numpy())

        test_loss = total_loss / len(test_loader.dataset)
        test_accuracy = total_correct / len(test_loader.dataset)
        test_precision, test_recall, test_f1 = calculate_metrics(y_true, y_pred)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
           f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, '
           f'Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}')

In [13]:
# Start training
train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=5, device=device)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPS: 4.454400 MFLOPs, Parameters: 0.031003 M


100%|██████████| 122/122 [01:20<00:00,  1.52it/s]


Epoch 1/5, Train Loss: 0.0107, Train Accuracy: 0.8819, Test Loss: 0.0046, Test Accuracy: 0.9464, Test Precision: 0.8942, Test Recall: 0.8948, Test F1: 0.8941


100%|██████████| 122/122 [01:15<00:00,  1.61it/s]


Epoch 2/5, Train Loss: 0.0027, Train Accuracy: 0.9706, Test Loss: 0.0017, Test Accuracy: 0.9866, Test Precision: 0.9738, Test Recall: 0.9738, Test F1: 0.9738


...

Epoch 3/5, Train Loss: 0.0017, Train Accuracy: 0.9801, Test Loss: 0.0014, Test Accuracy: 0.9897, Test Precision: 0.9820, Test Recall: 0.9821, Test F1: 0.9820


100%|██████████| 122/122 [01:15<00:00,  1.62it/s]


Epoch 4/5, Train Loss: 0.0011, Train Accuracy: 0.9887, Test Loss: 0.0012, Test Accuracy: 0.9897, Test Precision: 0.9889, Test Recall: 0.9889, Test F1: 0.9889


100%|██████████| 122/122 [01:16<00:00,  1.60it/s]


Epoch 5/5, Train Loss: 0.0012, Train Accuracy: 0.9869, Test Loss: 0.0014, Test Accuracy: 0.9866, Test Precision: 0.9868, Test Recall: 0.9868, Test F1: 0.9868
